
- overall goal:
    - specific ROI targetting w/ minimal campaigns
- what this means for accnt structure
    - for many variables we must split campaigns to target ROI
    - want to capture greatest amt of rps variation w/ 
        fewest # of campaigns
- 2 ways of approaching this
    1. minimize rps variation w/in a campaign
        - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
        - tried this out a little - but the computation was intensive which made it slow going
    2. maximize rps variation outside campaigns
    - i.e. after campaign split want to minimize rps variance w/in campaigns
        => i actually think this is eq to decision tree regression w/ split criterion MSE
        https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
    - mostly focused on this approach

- approaches I tried
1. rank splits by their correlation/covaraince/MI w/ `agg_rps`
    - tried ANOVA,pt.biserial,MI - had most success w/ MI
    - was dificult to compute b/c couldnt find methods that accepted sample weight
    - MI approach was promising but didnt go down that route
2. rank splits by intra-split agg_rps variance
    - had most success w/ this
    - tested out an approahc where I cluster the split on rps - had good results
    - think this is the mtd to use going forward
    TODO:
    - test fitting on general traffic sources?
    - test clustering on multiple days of rps data - 
        or rolling rps data or something
    - try out more granular aggs
    - is there some kind of metric which measures total variance
        and computes how much of that variance is captured by a split?

3. fit decision tree on rps data 
    - MSE criterion is apparently the same as minimizng inter split variance
    - computationally intensive
    - not 100% clear how to go from tree to campaign structure


- can do this by
    1. choosing what vars or tuples of vars to split campaigns on
    2. grouping similar buckets w/in those splits
- want to measure correlation/dependence of categorical split vars w/ cont rps
    - cat,cat metrics w/ binned rps
        chi
    - cont,cont metrcs w/ 1 hot encoded split vars
    - cat,cont metrics
- correlation metrics
    - MI:
        - sklearn.feature_selection.mi_regression
        - would need AFAICT to rresample input arrays
    - ANOVA:
        - scipy.stats.f_oneway
    - pt biserial
        - needs binary vars tho
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pointbiserialr.html
        - https://towardsdatascience.com/point-biserial-correlation-with-python-f7cd591bd3b1
- want most variation of rps w/ fewest camapaigns
- dont want to group similar buckets together - think its too complicated


TODO: 2021-05-20
  - test fitting on general traffic sources?
  - test clustering on multiple days of rps data - 
      or rolling rps data or something
  - try out more granular aggs
  - is there some kind of metric which measures total variance
      and computes how much of that variance is captured by a split?

Trevor: 2021-05-21
TODO:
- figure out how to minimize campaign # when writing back to taboola
    - want to make sure campaigns have sufficient traffic
- kw=(location,os,device)
- calc 30-day rps/kw
- can create distribution over the rps(kw) distribution
- 100 campaigns - 1 per percentile

TODO: 2021-05-24
- look into clustering each individual split variable

In [1]:
import sys
import re
import os
# detect if we are running from a `notebooks/*_shared` folder
# re.match("not.*shared",sys.path[0])
if sys.path[0].endswith("_shared"):
    sys.path[0] = "/".join(sys.path[0].split("/")[:-2])
assert sys.path[0].endswith("adtech")

from utils.env import load_env_from_aws
load_env_from_aws()

import pprint
from IPython.display import display as ipydisp    
import collections
import itertools
import datetime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from models.utils import wavg,get_wavg_by
from notebooks.aduriseti_shared.utils import *

NOW = datetime.datetime.now()
DAY = datetime.timedelta(days=1)

campaign_split_fields = dict(
    # traffic_source = ["traffic_source"],
    browser = ["browser"],
    operating_system = ["operating_system"],
    device = ["device"],
    # channel = ["channel"],
    # domain = ["domain"],
    product = ["product"],
    # keyword = ["keyword"],
    # campaign_id = ["campaign_id"],
    # landing_page = ["landing_page"],
    TOD = ["dayofweek","hourofday"],
    dma = ["dma"],
    state =["state",],
    location = ["state","dma"],
    
    dma_os=["dma", "operating_system"],
    dma_device=["dma", "device", ],
    dma_os_device=["dma", "operating_system", "device"],

    state_os=["state", "operating_system"],
    state_device=["state", "device", ],
    state_os_device=["state", "operating_system", "device"],

    location_os = ["state", "dma", "operating_system"],
    location_device=["state", "dma", "device", ],
    location_os_device = ["state", "dma", "operating_system","device"],
)

def get_wthresh(W,p):
    W = rps_df["sessions"].sort_values(ascending=False)
    Wsum = W.sum()
    cumsum = 0
    for wthresh in W:
        if cumsum > Wsum * p:
            break
        cumsum += wthresh
    return wthresh

start_date = NOW - 90*DAY
eval_date = NOW - 30*DAY
end_date = NOW

split2aggrps = {}
for split,split_cols in campaign_split_fields.items():
    print(split,split_cols)
    rps_df = agg_rps(start_date,end_date,None,traffic_source=TABOOLA,agg_columns=tuple(split_cols+["utc_dt"]))
    rps_df = translate_taboola_vals(rps_df)
    rps_df["split_on"] = split
    split2aggrps[split] = rps_df
    print(split,rps_df.shape)

Attempting to load environ from: `SM_ENV_BASE`
...Success!!
browser ['browser']


INFO:root:query: SELECT browser,utc_dt, COUNT(session_id)                    ... executed in 4.47 seconds
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


browser (94, 11)
operating_system ['operating_system']


INFO:root:query: SELECT operating_system,utc_dt, COUNT(session_id)           ... executed in 4.43 seconds


operating_system (340, 11)
device ['device']


...

location_os (17620, 11)
location_device ['state', 'dma', 'device']


INFO:root:query: SELECT state,dma,device,utc_dt, COUNT(session_id)           ... executed in 4.91 seconds


location_device (15343, 11)
location_os_device ['state', 'dma', 'operating_system', 'device']


INFO:root:query: SELECT state,dma,operating_system,device,utc_dt, COUNT(sessi... executed in 5.03 seconds


location_os_device (17881, 11)


In [117]:
import sklearn.preprocessing
import sklearn.cluster
import sklearn.tree

class AggRPSClust:
    def __init__(self,minclust=8):
        self.minclust = minclust
        self.split_size = None
    
    def fit(self,X,_):
        globals()["X"] = X
        assert X.index.names[-1] == "utc_dt"
        split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
        self.split_size = split_idx.drop_duplicates().__len__()
        if self.split_size <= self.minclust:
            self.ord_enc = sklearn.preprocessing.OrdinalEncoder(
                    handle_unknown="use_encoded_value",unknown_value=-1) \
                .fit(split_idx.drop_duplicates().values.reshape(-1,1))
            self.ord_enc.transform(split_idx.values.reshape(-1,1))
        else:
            self.aggX = X \
                .groupby(X.index.names[:-1]) \
                .agg({
                    "sessions": sum,
                    "rps": get_wavg_by(X,"sessions")
                })

            # nclust = MINCLUST
            nclust = max(self.minclust, int(np.log(self.split_size)))
            # nclust = max(MINCLUST,int(split_size ** 0.5))
            # print("nclust", nclust, split_size, np.log(split_size))
            
            self.aggX['clust'] = sklearn.cluster.KMeans(n_clusters=nclust) \
                            .fit_predict(self.aggX[["rps"]],sample_weight=self.aggX["sessions"])

        return self

    def transform(self,X):
        globals()["X"] = X
        assert X.index.names[-1] == "utc_dt"
        split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
        if self.split_size <= self.minclust:
            return self.ord_enc.transform(split_idx.values.reshape(-1,1))
        else:
            X["clust"] = 1
            X["clust"] *= self.aggX["clust"]
            return X["clust"]
        
    def fit_transform(self,X,_):
        raise NotImplementedError
        
class TreeRPSClust:
    def __init__(self):
        pass
    
    def fit(self,X,_):

        globals()["X"] = X
        
        Xdf = X .reset_index()[X.index.names].copy()
        ydf = X["rps"]
        wdf = X["sessions"]

        ipydisp(Xdf.isna().sum())

        mincnt = 100
        for c in Xdf.columns:
            too_few_I = Xdf.groupby(c).transform("count").iloc[:,0] < mincnt
            Xdf.loc[too_few_I,c] = np.NaN

        ipydisp(Xdf.isna().sum())
        Xdf = Xdf.fillna("")
        Xdf = Xdf.iloc[:,:-1]

        self.enc_1hot = sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore") .fit(Xdf)
        self.enc_features = [*self.enc_1hot.get_feature_names()]
        X = self.enc_1hot.transform(Xdf)
        print("|X|",X.shape)
        y = ydf.fillna(0)
        w = wdf

        self.clf = sklearn.tree.DecisionTreeRegressor(min_samples_leaf=30) \
                        .fit(X,y,sample_weight=wdf)
        print(sklearn.tree.export_text(self.clf,feature_names=self.enc_features))
        
        
        yhat = self.clf.predict(X)
        print("Tree RPS MAE:",(y - yhat).abs().mean())

        return self
        
    def transform(self,X):
        globals()["X"] = X
        Xdf = X .reset_index()[X.index.names]
        X = self.enc_1hot.transform(Xdf.fillna("").values[:,:-1])
        print("|X|",X.shape)
        return self.clf.apply(X)
        
# class AggRPSClustBad:
#     def __init__(self,minclust=8):
#         self.minclust = minclust
#         self.split_size = None
    
#     def fit(self,X,_):
#         globals()["X"] = X
#         assert X.index.names[-1] == "utc_dt"
#         split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
#         self.split_size = split_idx.drop_duplicates().__len__()
#         if self.split_size <= self.minclust:
#             self.ord_enc = sklearn.preprocessing.OrdinalEncoder(
#                     handle_unknown="use_encoded_value",unknown_value=-1) \
#                 .fit(split_idx.drop_duplicates().values.reshape(-1,1))
#             self.ord_enc.transform(split_idx.values.reshape(-1,1))
#         else:
#             # nclust = MINCLUST
#             nclust = max(self.minclust, int(np.log(self.split_size)))
#             # nclust = max(MINCLUST,int(split_size ** 0.5))
#             # print("nclust", nclust, split_size, np.log(split_size))
            
#             aggX = X \
#                 .groupby(X.index.names[:-1]) \
#                 .agg({
#                     "sessions": sum,
#                     "rps": get_wavg_by(X,"sessions")
#                 })
#             self.clusterer = sklearn.cluster.KMeans(n_clusters=self.minclust) \
#                 .fit(aggX[["rps"]],sample_weight=aggX["sessions"])

#         return self

#     def transform(self,X):
#         globals()["X"] = X
#         assert X.index.names[-1] == "utc_dt"
#         split_idx = X.reset_index()[X.index.names[:-1]] .apply(tuple,axis=1).apply(str)
#         if self.split_size <= self.minclust:
#             return self.ord_enc.transform(split_idx.values.reshape(-1,1))
#         else:
#             aggX = X \
#                 .groupby(X.index.names[:-1]) \
#                 .agg({
#                     "sessions": sum,
#                     "rps": get_wavg_by(X,"sessions")
#                 })
#             aggX["clust"] = self.clusterer \
#                 .predict(aggX[["rps"]],sample_weight=aggX["sessions"])
#             X["clust"] = 1
#             X["clust"] *= aggX["clust"]
#             return X["clust"]
        
#     def fit_transform(self,X,_):
#         raise NotImplementedError

# import sklearn.cluster
# def cluster_on(session_df,start_date,eval_date,end_date,col,CLUSTS=8,kpis=["rps","score","lps","rpl"],plot=True):
#     kpi_df = session_df.groupby([*col,"utc_dt"]) \
#         .agg({
#             "rps": get_wavg_by(session_df,"sessions"),
#             "score": get_wavg_by(session_df,"sessions"),
#             "lps": get_wavg_by(session_df,"sessions"),
#             "rpl": get_wavg_by(session_df,"sessions"),
#             "sessions": "sum",
#             "revenue": "sum"
#         })
#     print("|kpi_df|",kpi_df.shape)

#     col_uvals = kpi_df.reset_index()[col].drop_duplicates().values

#     date_range = pd.date_range(start_date,eval_date)
#     # kpi_tensor = np.stack(kpi_df.loc[tuple(uval),kpis] \
#     #                           .reindex(date_range).fillna(0) \
#     #                           .rolling(7).mean().fillna(0)
#     #                       for uval in col_uvals)
#     kpi_tensor = np.stack(kpi_df.loc[tuple(uval),kpis] \
#                               .reindex(date_range).fillna(0) \
#                               .rolling(7).mean().fillna(0)
#                           for uval in col_uvals)
#     kpi_tensor = kpi_tensor.transpose(2,0,1)
#     D, H, W = kpi_tensor.shape
    
#     if plot:
#         # i = (kpi_tensor > 0).sum(axis=1)[:,0].argmax()
#         # i = kpi_tensor.sum(axis=2)[0,:].argmax()
#         i = (kpi_tensor > 1e-3).sum(axis=2)[0].argmax()
#         plt.plot(kpi_tensor[0,i,:])
#         plt.show()

# #     import scipy.ndimage
# #     scipy.ndimage.gaussian_filter1d(X, sigma=sigma, axis=0)
# #     kpi_tensor = scipy.ndimage.gaussian_filter1d(kpi_tensor,sigma=7,axis=2)
    
# #     if plot:
# #         plt.plot(kpi_tensor[0,i,:])
# #         plt.show()
    
# #     mu = kpi_tensor.mean(axis=2).reshape(D, H, 1)
# #     std = kpi_tensor.std(axis=2).reshape(D, H, 1)
# #     kpi_tensor_norm = (kpi_tensor - mu) / std
# #     kpi_tensor_norm[np.isnan(kpi_tensor_norm)] = 0
# #     kpi_corr = (kpi_tensor_norm @ kpi_tensor_norm.transpose(0, 2, 1)) / W
# #     kpi_corr_df = pd.DataFrame(kpi_corr.mean(axis=0), columns=col_uvals, index=col_uvals)
# #     # assert np.abs(np.diag(loc_corr_df) - 1).max() < 1e-10

#     kpi_sqrt_tensor = kpi_tensor ** 0.5
#     kpi_corps = (kpi_sqrt_tensor @ kpi_sqrt_tensor.transpose(0, 2, 1))
#     kpi_coprs = np.log(kpi_corps)
#     kpi_corps = kpi_corps / kpi_corps.max(axis=2).max(axis=1).reshape(D,1,1)
#     corps_df = pd.DataFrame(kpi_corps.mean(axis=0), columns=col_uvals, index=col_uvals)

# #     clust = sklearn.cluster \
# #         .KMeans(n_clusters=CLUSTS) \
# #         .fit_predict(kpi_corr_df.values,sample_weight=kpi_df.groupby(col)["sessions"].sum().values)
    
#     clust = sklearn.cluster \
#         .KMeans(n_clusters=CLUSTS) \
#         .fit_predict(corps_df.values,sample_weight=kpi_df.groupby(col)["sessions"].sum())

# #     clust = sklearn.cluster \
# #         .AgglomerativeClustering(affinity="precomputed",n_clusters=CLUSTS,linkage='complete') \
# #         .fit_predict((1-kpi_corr_df.values) / 2)

# #     clust = sklearn.cluster \
# #         .AgglomerativeClustering(affinity="precomputed",n_clusters=CLUSTS,linkage='complete') \
# #         .fit_predict(-corps_df.values)
    
#     if plot:
#         for ci in range(CLUSTS):
#             clust_kpi_df = pd.concat(kpi_df.loc[tuple(uval)] for uval in col_uvals[clust==ci]) \
#                 .reset_index()
#             print(ci,"rps:",wavg(clust_kpi_df["rps"],clust_kpi_df["sessions"]))
#             ipydisp(clust_kpi_df[["sessions",'revenue']].sum())
#             clust_kpi_df \
#                 .groupby("utc_dt")["rps"] \
#                 .agg(get_wavg_by(clust_kpi_df,"sessions")) \
#                 .reindex(pd.date_range(eval_date-7*DAY,end_date)) \
#                 .fillna(0).rolling(7).mean() \
#                 .plot(label=ci,figsize=(15,5))
#         plt.legend()
#         plt.show()

#     clust_df = pd.DataFrame(clust,columns=["clust"])
#     clust_df[col] = col_uvals
#     clust_ser = pd.merge(session_df.reset_index()[col],clust_df,on=col,how="left")["clust"]
#     clust_ser.index = session_df.index
#     return clust_ser

from models.utils import wstd
def get_split_factor(rps_df):
    split_attr2unique_vals = {c: rps_df.index.unique(c) for c in rps_df.index.names[:-1]}
    _,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
    return rps_df.reset_index()[[*new_index_order[:-1],"clust"]].drop_duplicates().__len__()

In [118]:
perfD = []
for split,rps_df in split2aggrps.items():
    
    rps_df["lps"] = rps_df["num_leads"] / rps_df["sessions"]
    rps_df["rpl"] = rps_df["revenue"] / rps_df["num_leads"]
    rps_df["score"] = rps_df[["score_null_avg","score_adv_avg","score_supp_avg"]].sum(axis=1)
    rps_df["rps"] = rps_df["rps_avg"]
    fitI = rps_df.reset_index()['utc_dt'] < eval_date
    fitI.index = rps_df.index

    for clust_mtd in [AggRPSClust,TreeRPSClust]:
        clusterer = clust_mtd().fit(rps_df[fitI],None)
        rps_df["clust"] = -1
        rps_df.loc[fitI,"clust"] = clusterer.transform(rps_df[fitI])
        rps_df.loc[~fitI,"clust"] = clusterer.transform(rps_df[~fitI])

        agg_rps_df = rps_df \
            [~fitI] \
            .groupby(rps_df.index.names[:-1]) \
            .agg({
                "sessions": sum,
                "rps": get_wavg_by(rps_df[~fitI],"sessions")
            })
        clust_rps_df = rps_df \
            [~fitI] \
            .groupby("clust") \
            .agg({
                "sessions": sum,
                "rps": get_wavg_by(rps_df[~fitI],"sessions")
            })

        assert clust_rps_df["rps"].max() <= agg_rps_df["rps"].max()
        rps_wavg = wavg(agg_rps_df[["rps"]], agg_rps_df["sessions"])
        rps_clust_wavg = wavg(clust_rps_df[["rps"]], clust_rps_df["sessions"])
        assert all((rps_wavg - rps_clust_wavg).abs() < 1e-2), (rps_wavg, rps_clust_wavg)


        perfd = {
            "clust_mtd": clusterer.__class__.__name__,
            "clusterer": clusterer,
            "split": split,
            "fit_shape": agg_rps_df.shape,
            "clust_shape": clust_rps_df.shape,
            "split_variance": wstd(agg_rps_df["rps"], agg_rps_df["sessions"]),
            "cluster_variance": wstd(clust_rps_df["rps"], clust_rps_df["sessions"]),
            # wstd(rps_df["rps_avg"],rps_df["sessions"])
            "clustered_split_factor": get_split_factor(rps_df),
        }
        perfD.append(perfd)
        pprint.pprint(perfd)
        ipydisp(clust_rps_df)

perfdf = pd.DataFrame(perfD)
ipydisp(perfdf)

{'clust_mtd': 'AggRPSClust',
 'clust_shape': (2, 2),
 'cluster_variance': 0.008213534619983488,
 'clustered_split_factor': 2,
 'clusterer': <__main__.AggRPSClust object at 0x7f4463921130>,
 'fit_shape': (2, 2),
 'split': 'browser',
 'split_variance': 0.008213534619983488}


,sessions,rps
clust,,
0.0,7,0.00000
1.0,25000,0.49099


browser    0
utc_dt     0
dtype: int64

browser    64
utc_dt     64
dtype: int64

|X| (64, 1)
|--- value: [0.42]

Tree RPS MAE: 0.19436372098347118
|X| (64, 1)
|X| (30, 1)
{'clust_mtd': 'TreeRPSClust',
 'clust_shape': (1, 2),
 'cluster_variance': 0.0,
 'clustered_split_factor': 1,
 'clusterer': <__main__.TreeRPSClust object at 0x7f446369fcd0>,
 'fit_shape': (2, 2),
 'split': 'browser',
 'split_variance': 0.008213534619983488}


...

state               0
dma                 0
operating_system    0
device              0
utc_dt              0
dtype: int64

state                681
dma                 9355
operating_system      56
device                 0
utc_dt               278
dtype: int64

|X| (12079, 71)
|--- x0_TX <= 0.50
|   |--- x2_Windows <= 0.50
|   |   |--- x0_IL <= 0.50
|   |   |   |--- x0_WI <= 0.50
|   |   |   |   |--- x2_Mac OS X <= 0.50
|   |   |   |   |   |--- x0_VA <= 0.50
|   |   |   |   |   |   |--- x1_524 <= 0.50
|   |   |   |   |   |   |   |--- x0_PA <= 0.50
|   |   |   |   |   |   |   |   |--- x0_CA <= 0.50
|   |   |   |   |   |   |   |   |   |--- x0_NC <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- x3_DESK <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 7
|   |   |   |   |   |   |   |   |   |   |--- x3_DESK >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 4
|   |   |   |   |   |   |   |   |   |--- x0_NC >  0.50
|   |   |   |   |   |   |   |   |   |   |--- value: [0.10]
|   |   |   |   |   |   |   |   |--- x0_CA >  0.50
|   |   |   |   |   |   |   |   |   |--- value: [0.23]
|   |   |   |   |   |   |   |--- x0_PA >  0.50
|   |   |   |   |   |   |   |   |--- x1_ <= 0.50
|   |   | 

,sessions,rps
clust,,
15,461,0.162603
16,35,0.000000
17,29,0.000000
18,53,0.000000
33,171,0.662807
...,...,...
229,1723,0.592565
230,99,0.936566
231,536,0.613526


,clust_mtd,clusterer,split,fit_shape,clust_shape,split_variance,cluster_variance,clustered_split_factor
0,AggRPSClust,<__main__.AggRPSClust object at 0x7f4463921130>,browser,"(2, 2)","(2, 2)",0.008214,0.008214,2
1,TreeRPSClust,<__main__.TreeRPSClust object at 0x7f446369fcd0>,browser,"(2, 2)","(1, 2)",0.008214,0.000000,1
2,AggRPSClust,<__main__.AggRPSClust object at 0x7f4463f9a070>,operating_system,"(6, 2)","(6, 2)",0.081340,0.081340,6
3,TreeRPSClust,<__main__.TreeRPSClust object at 0x7f4460b42e80>,operating_system,"(6, 2)","(1, 2)",0.081340,0.000000,1
4,AggRPSClust,<__main__.AggRPSClust object at 0x7f4460b42400>,device,"(3, 2)","(3, 2)",0.087784,0.087784,3
5,TreeRPSClust,<__main__.TreeRPSClust object at 0x7f4463552f10>,device,"(3, 2)","(1, 2)",0.087784,0.000000,1
6,AggRPSClust,<__main__.AggRPSClust object at 0x7f44637279a0>,product,"(2, 2)","(2, 2)",0.003104,0.003104,2
7,TreeRPSClust,<__main__.TreeRPSClust object at 0x7f446094f790>,product,"(2, 2)","(1, 2)",0.003104,0.000000,1
8,AggRPSClust,<__main__.AggRPSClust object at 0x7f44638a8f70>,TOD,"(522, 2)","(7, 2)",0.371523,0.051615,44
9,TreeRPSClust,<__main__.TreeRPSClust object at 0x7f4463a9ee80>,TOD,"(522, 2)","(7, 2)",0.371523,0.076318,7


In [52]:
X

revenue  sessions  num_leads  \
browser                     utc_dt                                        
Microsoft Internet Explorer 2021-04-05     0.00000         1        0.0   
                            2021-04-21     0.00000         1        0.0   
                            2021-05-12     0.00000         1        0.0   
Netscape                    2021-03-30  1074.36342      2047       87.0   
                            2021-03-31   810.46536      1619       69.0   
...                                            ...       ...        ...   
                            2021-05-25   629.52000      1733       50.0   
                            2021-05-26  1444.49411      2225       84.0   
                            2021-05-27  1236.42595      1593       65.0   
                            2021-05-28   579.83584      1031       41.0   
                            2021-05-29     6.13000         8        1.0   

                                         lps_avg    rpl_avg   rps_avg  \
browser                     utc_dt                                      
Microsoft Internet Explorer 2021-04-05  0.000000   0.000000  0.000000   
                            2021-04-21  0.000000   0.000000  0.000000   
                            2021-05-12  0.000000   0.000000  0.000000   
Netscape                    2021-03-30  0.042501  12.349005  0.524848   
                            2021-03-31  0.042619  11.745875  0.500596   
...                                          ...        ...       ...   
                            2021-05-25  0.028852  12.590400  0.363254   
                            2021-05-26  0.037753  17.196358  0.649211   
                            2021-05-27  0.040804  19.021938  0.776162   
                            2021-05-28  0.039767  14.142338  0.562401   
                            2021-05-29  0.125000   6.130000  0.766250   

                                        score_null_avg  score_adv_avg  \
browser                     utc_dt                                      
Microsoft Internet Explorer 2021-04-05        0.000000       0.000000   
                            2021-04-21        0.000000       0.000000   
                            2021-05-12        0.000000       0.000000   
Netscape                    2021-03-30        0.021933       0.000000   
                            2021-03-31        0.019494       0.000000   
...                                                ...            ...   
                            2021-05-25        0.004150       0.004047   
                            2021-05-26        0.001600       0.007194   
                            2021-05-27        0.004500       0.005127   
                            2021-05-28        0.001700       0.002529   
                            2021-05-29        0.000000       0.000000   

                                        score_supp_avg  int_ix split_on  \
browser                     utc_dt                                        
Microsoft Internet Explorer 2021-04-05        0.000000       0  browser   
                            2021-04-21        0.000000       1  browser   
                            2021-05-12        0.000000       2  browser   
Netscape                    2021-03-30        0.000000       6  browser   
                            2021-03-31        0.000000       7  browser   
...                                                ...     ...      ...   
                            2021-05-25        0.016011      62  browser   
                            2021-05-26        0.018463      63  browser   
                            2021-05-27        0.019830      64  browser   
                            2021-05-28        0.019071      65  browser   
                            2021-05-29        0.000000      66  browser   

                                             rps  clust       lps        rpl  \
browser                     utc_dt                                             
Microsoft Internet Explorer 202

In [119]:
split,clusterer = perfdf.loc[31,["split","clusterer"]]
rps_df = split2aggrps[split]
rps_df["lps"] = rps_df["num_leads"] / rps_df["sessions"]
rps_df["rpl"] = rps_df["revenue"] / rps_df["num_leads"]
rps_df["score"] = rps_df[["score_null_avg","score_adv_avg","score_supp_avg"]].sum(axis=1)
rps_df["rps"] = rps_df["rps_avg"]
fitI = rps_df.reset_index()['utc_dt'] < eval_date
fitI.index = rps_df.index

rps_df["clust"] = -1
rps_df.loc[fitI,"clust"] = clusterer.transform(rps_df[fitI])
rps_df.loc[~fitI,"clust"] = clusterer.transform(rps_df[~fitI])

split_attr2unique_vals = {index_col: rps_df.index.unique(index_col) 
                          for index_col in rps_df.index.names[:-1]}
_,new_index_order = zip(*sorted((V.__len__(),c) for c,V in split_attr2unique_vals.items()))
rps_df = rps_df.reset_index()
campaign_df = rps_df \
    .groupby([*new_index_order[:-1], "clust"]) \
    .agg({
        "sessions": sum,
        "rps_avg": get_wavg_by(rps_df,"sessions"),
        new_index_order[-1]: lambda seq: tuple(set(seq))
    }) \
    .sort_index()

|X| (10593, 54)
|X| (4750, 54)


sessions   rps_avg  \
device state clust                       
DESK   AK    60          214  0.246028   
       AL    107        1394  0.432337   
       AR    96          790  0.463445   
       AZ    60          929  0.373940   
       CA    90         2541  0.281119   
...                      ...       ...   
TBLT   VA    147          24  0.000000   
       VT    15            3  0.000000   
       WA    15            2  0.000000   
       WI    29           20  0.000000   
       WV    15            5  0.000000   

                                                                  dma  
device state clust                                                     
DESK   AK    60                                       (747, 745, 743)  
       AL    107        (524, 686, 630, 711, 698, 691, 575, 606, 522)  
       AR    96              (640, 670, 647, 612, 628, 619, 693, 734)  
       AZ    60                                       (753, 771, 789)  
       CA    90     (862, 804, 753, 803, 807, 855, 813, 828, 802, ...  
...                                                               ...  
TBLT   VA    147                       (556, 584, 511, 573, 544, 531)  
       VT    15                                            (506, 523)  
       WA    15                                                (819,)  
       WI    29                             (613, 617, 705, 658, 669)  
       WV    15                                       (554, 598, 564)  

[176 rows x 3 columns]

In [123]:
# from pkg_resources import resource_filename as rscfn
# rscfn(__name__,"campaign.csv")

'campaign.csv'

In [124]:
assert campaign_df["sessions"].sum() == rps_df["sessions"].sum()
camp_rps_wavg = wavg(campaign_df["rps_avg"],campaign_df["sessions"])
fit_rps_wavg = wavg(rps_df["rps_avg"], rps_df["sessions"])
assert abs(camp_rps_wavg - fit_rps_wavg) < 1e-5

dma
device state                                                   
DESK   AK                                       (747, 745, 743)
       AL         (524, 686, 630, 711, 698, 691, 575, 606, 522)
       AR              (640, 670, 647, 612, 628, 619, 693, 734)
       AZ                                       (753, 771, 789)
       CA     (862, 804, 753, 803, 807, 855, 813, 828, 802, ...
...                                                         ...
TBLT   VA                        (556, 584, 511, 573, 544, 531)
       VT                                            (506, 523)
       WA                                                (819,)
       WI                             (613, 617, 705, 658, 669)
       WV                                       (554, 598, 564)

[152 rows x 1 columns]

In [127]:
camps = []
for idx,r in campaign_df.iterrows():
    camp = {
        "sessions_60d": r["sessions"], 
        "rps_avg_60d": r["rps_avg"]
    }
    for field,val in zip(new_index_order[:-1],idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "includes": r[last_field]
    }
    camps.append(camp)

excl_campaign_df = campaign_df.groupby([*new_index_order[:-1]]) \
    .agg({
        new_index_order[-1]: tuple
    })
def flatten(M):
    return tuple(el for r in M for el in r)
excl_campaign_df[new_index_order[-1]] = excl_campaign_df[new_index_order[-1]] \
    .apply(flatten)
for idx, r in excl_campaign_df.iterrows():
    camp = {}
    for field, val in zip(new_index_order[:-1], idx):
        camp[field] = {"includes": val}
    last_field = new_index_order[-1]
    camp[last_field] = {
        "excludes": r[last_field]
    }
    camps.append(camp)

camp_attr_df = pd.DataFrame(camps)
camp_attr_df.to_csv("campaign_dump.csv")

In [128]:
camp_attr_df

,sessions_60d,rps_avg_60d,device,state,dma
0,214.0,0.246028,{'includes': 'DESK'},{'includes': 'AK'},"{'includes': ('747', '745', '743')}"
1,1394.0,0.432337,{'includes': 'DESK'},{'includes': 'AL'},"{'includes': ('524', '686', '630', '711', '698..."
2,790.0,0.463445,{'includes': 'DESK'},{'includes': 'AR'},"{'includes': ('640', '670', '647', '612', '628..."
3,929.0,0.373940,{'includes': 'DESK'},{'includes': 'AZ'},"{'includes': ('753', '771', '789')}"
4,2541.0,0.281119,{'includes': 'DESK'},{'includes': 'CA'},"{'includes': ('862', '804', '753', '803', '807..."
...,...,...,...,...,...
323,NaN,NaN,{'includes': 'TBLT'},{'includes': 'VA'},"{'excludes': ('556', '584', '511', '573', '544..."
324,NaN,NaN,{'includes': 'TBLT'},{'includes': 'VT'},"{'excludes': ('506', '523')}"
325,NaN,NaN,{'includes': 'TBLT'},{'includes': 'WA'},"{'excludes': ('819',)}"
326,NaN,NaN,{'includes': 'TBLT'},{'includes': 'WI'},"{'excludes': ('613', '617', '705', '658', '669')}"


In [131]:
import pytaboola
# hc creds
hccreds = {
  "client_id": "3cc90f549fd44a9fbe36f70aa6e7cfde",
  "client_secret": "eaa74a94108648579e65167bf276c3a1",
}
# pivot creds
pivotcreds = {
  "client_id": "31d7f068eb3d43acb85b5ad64eef2ca9",
  "client_secret": "c5010044df194010817e76edac0fd426",
}

from pytaboola import TaboolaClient
client = TaboolaClient(**hccreds)
client.token_details

import requests
import jmespath
TABOOLA_BASE = "https://backstage.taboola.com/backstage/api/1.0"
resp = requests.get(
    f"{TABOOLA_BASE}/resources/campaigns_properties/operating_systems",
    headers=client.authorization_header)
taboola_os = jmespath.search('results[].name',resp.json(),)

resp = requests.get(
    f"{TABOOLA_BASE}/resources/campaigns_properties/platforms",
    headers=client.authorization_header)
taboola_platforms = jmespath.search('results[].name', resp.json(),)

resp = requests.get(
    f"{TABOOLA_BASE}/resources/countries/us/dma",
    headers=client.authorization_header)
taboola_dmas = jmespath.search('results[].name',resp.json(),)

resp = requests.get(
    f"{TABOOLA_BASE}/resources/countries/us/regions",
    headers=client.authorization_header)
taboola_states = jmespath.search('results[].name',resp.json(),)

/home/amal/anaconda3/envs/hc/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
# hc creds
hccreds = {
  "client_id": "3cc90f549fd44a9fbe36f70aa6e7cfde",
  "client_secret": "eaa74a94108648579e65167bf276c3a1",
}
# pivot creds
pivotcreds = {
  "client_id": "31d7f068eb3d43acb85b5ad64eef2ca9",
  "client_secret": "c5010044df194010817e76edac0fd426",
}

from pytaboola import TaboolaClient
client = TaboolaClient(**hccreds)
client.token_details

from pytaboola.services import AccountService
from pytaboola.services import CampaignService

import itertools
import tqdm
import pandas as pd
import os
from pkg_resources import resource_filename as rscfn


acct_service = AccountService(client)
accnts = acct_service.list()["results"]
NETWORK_ACCNT_ID = "healthcareinc-network"
TEST_ACCNT_ID = "healthcareinc-sc2"
O65_ACCNT_ID = "taboolaaccount-rangaritahealthcarecom"
id2accnt = {a["account_id"]: a for a in accnts}

def accnt_camps(accnt):
  camp_service = CampaignService(client, accnt["account_id"])
  return camp_service.list()
aid2cid2camp = {}
for aid,a in tqdm.tqdm(id2accnt.items()):
  cid2camp = {c["id"]: c for c in accnt_camps(a)}
  aid2cid2camp[aid] = cid2camp

import json
json.dump(aid2cid2camp, open("camps.json", "w"))
O65_accnt_camps = accnt_camps(id2accnt[O65_ACCNT_ID])
print("|065_accnt_camps|:",len(O65_accnt_camps))
#%%
import itertools
cross = itertools.product
import jmespath
get = jmespath.search

active_camps = get(
  "*.*[] | [?is_active]",
  aid2cid2camp,
)
print("|active campaigns|:",len(active_camps))
#%%
o65_camp_svc = CampaignService(client,O65_ACCNT_ID)
test_camp_svc = CampaignService(client,TEST_ACCNT_ID)
test_accnt_camps = aid2cid2camp[TEST_ACCNT_ID].values()
test_accnt_camp_names = {c["name"] for c in test_accnt_camps}
print("|test_accnt_camps|:",len(test_accnt_camps))
def copy_campaign_base(camp,to=TEST_ACCNT_ID):
  taboola_req_fields = [
    "name","branding_text","cpc","spending_limit",
    "spending_limit_model","marketing_objective"]
  camp_svc = CampaignService(client,to)
  camp_base = {f: camp[f] for f in taboola_req_fields}
  camp_copy = camp_svc.create(**camp_base)
  print(f"created base for `{camp['name']}` in account `{to}` w/ id:", camp_copy["id"])
  camp_copy = camp_svc.update(camp_copy["id"], is_active=False)
  assert camp_copy["is_active"] == False
  print("...succesfully deactivated newly created campaign")
  return camp_copy
for camp in active_camps:
  # skip already created test account campaigns
  if camp["name"] in test_accnt_camp_names: 
    print(f"Campaign `{camp['name']}` already exists in account `{TEST_ACCNT_ID}`")
  else:
    camp_copy = copy_campaign_base(camp,to=TEST_ACCNT_ID)
#%%
#%%
[c["status"] for c in test_accnt_camps]
#%%
def get_camp_by_name(name,accnt=TEST_ACCNT_ID):
  # TODO: figure out some way to avoid having to refetch all campaigns
  #       => will be able to use catalog tables eventually
  camp_svc = CampaignService(client, accnt)
  camps = camp_svc.list()
  match_camps = [c for c in camps if c["name"] == name]
  assert len(match_camps) == 1, (name,len(match_camps))
  return match_camps[0]
import pytaboola
pytaboola.errors.TaboolaError
import traceback
import tqdm
field_update_attempts = {}
field_update_errors = {}
def update_fields(camp,accnt,camp_id):    
  camp_svc = CampaignService(client, accnt)
  for k,attr in tqdm.tqdm([*camp.items()]):
    # print(f"attempting to copy over field `{k}`...")
    try:
      camp_svc.update(camp_id,**{k: attr})
      field_update_attempts.setdefault(k,[]).append(1)
      # print("...Sucess!")
    except (Exception,pytaboola.errors.TaboolaError) as ex:
      errd = {
        "ex": ex,
        "type": type(ex),
        "str": str(ex),
        "tb": traceback.format_exc()
      }
      # print(f"Failed w/ {type(ex)}:{ex}\n{traceback.format_exc()}")
      # print(f"Failed w/ {type(ex)}:{ex}")
      field_update_attempts.setdefault(k, []).append(0)
      field_update_attempts.setdefault(k, []).append(errd)

for camp in active_camps:
  dest_camp = get_camp_by_name(camp["name"])
  dest_id = dest_camp["id"]
  print("UPDATEING",camp["name"],dest_id)
  update_fields(camp,TEST_ACCNT_ID,dest_id)

update_attempt_df = pd.DataFrame(field_update_attempts)
update_attempt_df.mean(axis=0)


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import sklearn.feature_selection
import sklearn.metrics
from matplotlib import pyplot as plt
import sklearn.preprocessing

requires_campaign_split = [
    # "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]

for split in requires_campaign_split:
    rps_df = split2aggrps[split] .reset_index()
    Xs = rps_df \
        .apply(lambda r: [r["int_ix"]]*int(r["sessions"]), axis=1)
    X = np.concatenate(Xs.values).reshape(-1,1)
    ys = rps_df \
        .apply(lambda r: [r["rps_avg"]]*int(r["sessions"]),axis=1)
    y = np.concatenate(ys.values)
    # print(y.min(),np.quantile(y, 0.5),y.max())
    y = y > y.mean()
    # y = np.concatenate(ys.values).reshape(-1,1)
    # y = sklearn.preprocessing.KBinsDiscretizer(n_bins=2,encode="ordinal") \
    #     .fit_transform(y).reshape(-1)
    mi = sklearn.feature_selection.mutual_info_regression(X,y,discrete_features=True)
    print(split,mi,y.mean(),rps_df.shape)

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

requires_campaign_split = [
    "browser",
    "operating_system",
    "device",
    # "channel", # not actually sure what this is
    # "domain",
    "product",
    # "landing_page",
    "location",
]
combined_rps_df = pd.concat([df.reset_index() for df in split2aggrps.values()])
splitI = combined_rps_df["split_on"].isin(requires_campaign_split)
density = scipy.stats.gaussian_kde(
    dataset=combined_rps_df.loc[splitI,"rps_avg"],
    weights=combined_rps_df.loc[splitI, "sessions"],
    # bw_method="scott",
    # bw_method="silverman",
    # bw_method=0.1,
)
xs = np.linspace(0,2,100)
plt.plot(xs,density(xs))

In [ ]:
for split in requires_campaign_split:
    splitI = combined_rps_df["split_on"] == split
    density = scipy.stats.gaussian_kde(
        dataset=combined_rps_df.loc[splitI, "rps_avg"],
        weights=combined_rps_df.loc[splitI, "sessions"])
    xs = np.linspace(0, 2, 100)
    plt.plot(xs, density(xs))
    plt.title(split)
    plt.show()